In [43]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
model = "gpt-4o-mini"

In [44]:
from openai import OpenAI

client = OpenAI()

In [45]:
messages = [
    {"role": "system", "content": (
        "You are a fact checker. Verify the validity of the sentence provided by the user, given a reference. You must return a response in JSON format:"
        "{'isFactTrue': <true or false>, 'explanation': <explanation> ")
    },
    {"role": "user", "content": "White sand is made from fish poop https://oceanservice.noaa.gov/facts/sand.html"}
]

response = client.chat.completions.create(
  model=model,
  messages=messages,
  response_format={"type": "json_object"},
)

print(response.choices[0].message.content)

{
  "isFactTrue": false,
  "explanation": "White sand is primarily made up of small fragments of coral, shells, and other carbonate materials, rather than fish poop. While fish contribute to the marine ecosystem and some organic materials do get recycled into sediments, the claim that white sand is made specifically from fish poop is not accurate."
}


In [46]:
import requests
from bs4 import BeautifulSoup

def get_text_from_url(url):
    # response = requests.get(url)
    # if response.status_code == 200:
    #     soup = BeautifulSoup(response.content, 'html.parser')
    #     return soup.get_text()
    # else:
    #     return "Error: Unable to retrieve content"
    return '''The famous white-sand beaches of Hawaii, for example, actually come from the poop of parrotfish. The fish bite and scrape algae off of rocks and dead corals with their parrot-like beaks, grind up the inedible calcium-carbonate reef material (made mostly of coral skeletons) in their guts, and then excrete it as sand.'''

In [47]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_text_from_url",
            "description": "Get the text contents of a url",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url",
                    },
                },
                "required": ["url"],
            },
        },
    }
]

response = client.chat.completions.create(
  model=model,
  messages=messages,
  tools=tools,
  response_format={"type": "json_object"},
)

In [48]:
print(response)

ChatCompletion(id='chatcmpl-9ruaRBCDmSw2mu2AqCwHpUHyXipoA', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GeVV69N5H7OXwKstSpIfqqCX', function=Function(arguments='{"url":"https://oceanservice.noaa.gov/facts/sand.html"}', name='get_text_from_url'), type='function')]))], created=1722635931, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0f03d4f0ee', usage=CompletionUsage(completion_tokens=28, prompt_tokens=120, total_tokens=148))


In [49]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls


In [50]:
import json

if tool_calls:
    available_functions = {
        "get_text_from_url": get_text_from_url,
    }
    messages.append(response_message)  # extend conversation with assistant's reply
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            url=function_args.get("url"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        ) 
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    ) 

print(second_response)

ChatCompletion(id='chatcmpl-9ruaSSLuMYjzxuaFx13FUr3RDuad6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "isFactTrue": true,\n  "explanation": "The sentence is valid; white sand, particularly in Hawaii, is indeed made from the excrement of parrotfish, which grind and process calcium-carbonate materials from coral and then excrete it as sand."\n}', role='assistant', function_call=None, tool_calls=None))], created=1722635932, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_9b0abffe81', usage=CompletionUsage(completion_tokens=58, prompt_tokens=192, total_tokens=250))


In [51]:
print(second_response.choices[0].message.content)

{
  "isFactTrue": true,
  "explanation": "The sentence is valid; white sand, particularly in Hawaii, is indeed made from the excrement of parrotfish, which grind and process calcium-carbonate materials from coral and then excrete it as sand."
}
